In [1]:
import os
import sys
#!{sys.executable} -m pip install statsmodels


import numpy as np
import pandas as pd
import sys; sys.path
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multitest import multipletests as fdr
from matplotlib import colors

import warnings
warnings.filterwarnings('ignore')

In [2]:
results_dir   = 'results_directory_path'

ABCD_clin_labels = ['AnxDep', 'WithDep', 'Somatic', 'Social', 'Thought', 'Attention',
                    'RuleBreak', 'Aggresive', 'Internal', 'External', 'TotProb', 'Depress',
                    'AnxDiscord', 'SomaticPr', 'ADHD', 'Opposit', 'Conduct', 'Sluggish', 
                    'OCD', 'Stress']


#load in all of the prediction accuracy data for the true and the null models
rep = 100
n_beh = 20
corr_mm = np.zeros([n_beh,rep,n_beh])
corr_mf = np.zeros([n_beh,rep,n_beh])
corr_fm = np.zeros([n_beh,rep,n_beh])
corr_ff = np.zeros([n_beh,rep,n_beh])


null_rep = 1000
null_corr_mm = np.zeros([n_beh,null_rep,n_beh])
null_corr_mf = np.zeros([n_beh,null_rep,n_beh])
null_corr_fm = np.zeros([n_beh,null_rep,n_beh])
null_corr_ff = np.zeros([n_beh,null_rep,n_beh])

for i in range(len(ABCD_clin_labels)):
    
    corr_mm[i,:,:] = np.load(results_dir + '/fc_crossbehav_corr_mm_' + ABCD_clin_labels[i] + '.npy')
    corr_mf[i,:,:] = np.load(results_dir + '/fc_crossbehav_corr_mf_' + ABCD_clin_labels[i] + '.npy')
    corr_fm[i,:,:] = np.load(results_dir + '/fc_crossbehav_corr_fm_' + ABCD_clin_labels[i] + '.npy')
    corr_ff[i,:,:] = np.load(results_dir + '/fc_crossbehav_corr_ff_' + ABCD_clin_labels[i] + '.npy')
    
    null_corr_mm[i,:,:] = np.load(results_dir + '/fc_crossbehav_null_corr_mm_' + ABCD_clin_labels[i] + '.npy')
    null_corr_mf[i,:,:] = np.load(results_dir + '/fc_crossbehav_null_corr_mf_' + ABCD_clin_labels[i] + '.npy')
    null_corr_fm[i,:,:] = np.load(results_dir + '/fc_crossbehav_null_corr_fm_' + ABCD_clin_labels[i] + '.npy')
    null_corr_ff[i,:,:] = np.load(results_dir + '/fc_crossbehav_null_corr_ff_' + ABCD_clin_labels[i] + '.npy')



In [3]:
#compute average prediction accuracy
avg_corr_mm = np.mean(corr_mm, axis=1)
avg_corr_mf = np.mean(corr_mf, axis=1)
avg_corr_fm = np.mean(corr_fm, axis=1)
avg_corr_ff = np.mean(corr_ff, axis=1)

corr_m = np.hstack([avg_corr_mm, avg_corr_mf])
corr_f = np.hstack([avg_corr_fm, avg_corr_ff])

corr = np.vstack([corr_m, corr_f])

#np.savetxt((results_dir + '/summary_corr_corrected_5260.csv'), corr, delimiter=',')


In [4]:
#function to compare perforance between a model and its null 
def get_null_p(x,null):
    pval = 1-np.mean(x-null>=0)
    
    return pval

In [5]:
#compare true point estimate (mean) to entire null distribution
p_mm = np.zeros([n_beh,n_beh])
p_mf = np.zeros([n_beh,n_beh])
p_fm = np.zeros([n_beh,n_beh])
p_ff = np.zeros([n_beh,n_beh])


#get p vals for all models
for i in range(n_beh):
    for j in range(n_beh):
        p_mm[i,j] = get_null_p(np.mean(corr_mm[i,:,j]), null_corr_mm[i,:,j])
        p_mf[i,j] = get_null_p(np.mean(corr_mf[i,:,j]), null_corr_mf[i,:,j])
        p_fm[i,j] = get_null_p(np.mean(corr_fm[i,:,j]), null_corr_fm[i,:,j])
        p_ff[i,j] = get_null_p(np.mean(corr_ff[i,:,j]), null_corr_ff[i,:,j])
             
    
    #p_mm[i,:] = fdr(p_mm[i,:], alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)[1]
    #p_mf[i,:] = fdr(p_mf[i,:], alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)[1]
    #p_fm[i,:] = fdr(p_fm[i,:], alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)[1]
    #p_ff[i,:] = fdr(p_ff[i,:], alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)[1]
    

p_m = np.hstack([p_mm, p_mf])
p_f = np.hstack([p_fm, p_ff])

p = np.vstack([p_m, p_f])

#correct for multiple comparisons across all prediction accuracies
p_corr = fdr(p.ravel(), alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)[1]
p_corr = np.reshape(p_corr, (34, 34))


#np.sum(p_corr<(0.05))
#save accuracy data and corrected p values
np.savetxt((results_dir + '/summary_corr.csv'), corr, delimiter=',')
np.savetxt((results_dir + '/summary_p_corr.csv'), p_corr, delimiter=',')

161